# Initialize

In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import seaborn as sns
import collections as col
from itertools import combinations


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Beginning
If the cells are collapsed, just press run all, dont have to expand since they are very long. Also just to avoid any modifications :)

In [73]:
#---------------------------------------------------#
#n=size of matrix(ie. if you need a nxn matrix)     
#prob=probability of each block having a mine         
#ite= how many times you want to run the experiment 


n = 4                                              
prob = 0.9                                       
ite=1000 
p_m = [39,
104,
26,
346,
251] #mine numbers entered here
m_list = [10,
11,
12,
13,
14] # corresponding p(m)*1000 here


#---------------------------------------------------#

In [74]:


'''
First cell gives the number of times each mine appears in 1000 runs and P(A_i) for a given mine probability
Second cell gives number of paths
Third cell calculates E[X] 
'''




#mine_num_list = []
#mine_freq = {}
exp_grids = []
#g is an array with len n*n-2 wich holds values 1 or 0, And mine_num_list is a list of number of mines in each of the 1000 runs eg[1,1,1,1,2,2,2,0,0,0,0...(1000elements)]
for i in range(len(p_m)):
    for j in range(p_m[i]):
        mine_num = 0
        g = np.array([1] * ((n*n)- m_list[i] - 2) + [0]*(m_list[i]))
        np.random.shuffle(g)
        exp_gr = np.concatenate(([1],g,[1]))
        exp_gr = np.reshape(exp_gr, (n,n))
        exp_grids.append(exp_gr)
    #for i in g:
        #if i==0:
            #mine_num+=1
    #mine_num_list.append(mine_num)

#for items in mine_num_list:
    #mine_freq[items] = (mine_num_list.count(items))
      
#for key, value in mine_freq.items():
    #print ("% d : %d"%(key, value))


# All functions

In [75]:
#grid, arrays of all arrangements of a given number of mines. Size=n
def gridarray(size,mines):
    for positions in combinations(range(size), mines):
        p = [1] * (size)

        for i in positions:
            p[i] = 0

        yield p

#it reshapes all arrays from p to give matrices of all cofigurations for a particular numebr of mines
def grids(n,m):
    gridlist = []
    g = list(gridarray(n*n -2 , m))
    for gr in g:
        gr = np.concatenate(([1],gr,[1]))
        gr = np.reshape(gr, (n,n))
        gridlist.append(gr)
        
    return gridlist

#saves indices of all mines in the grid, currently not in use
def findmines(maze):
    result = []
    for i in range(len(maze)):
        for j in range(len(maze[i])):
            if maze[i][j] == 0:
                result.append([i, j])
    return result


def isValidCell(x, y, maze):
    return not (x < 0 or y < 0 or x >= N or y >= N)
 

def countPaths(maze, x, y, visited, count,templist,dataframe):
    global count_mine, count_block
    
    # if destination (bottom-rightmost cell) is found,
    # increment the path count
    if x == N - 1 and y == N - 1:
        #templist.append([N-1,N-1])
        #duplicate templist here
        dataframe.append(['s'])
        return count + 1
    
 
    # mark the current cell as visited
    visited[x][y] = True
    #print("appended ",x,y)
    #append element here, which can later be removed if it does not satisfy below if statements, we can remove element at backtracking stage.
    #templist.append([x,y])
 
    # if the current cell is a valid and open cell
    if isValidCell(x, y,maze) and maze[x][y] == 1:
 
        # go down `(x, y)` ——> `(x + 1, y)`
        if x + 1 < N and not visited[x + 1][y]:
            count = countPaths(maze, x + 1, y, visited, count,templist,dataframe)
 
        # go up `(x, y)` ——> `(x - 1, y)`
        if x - 1 >= 0 and not visited[x - 1][y]:
            count = countPaths(maze, x - 1, y, visited, count,templist,dataframe)

        # go right `(x, y)` ——> `(x, y + 1)`
        if y + 1 < N and not visited[x][y + 1]:
            count = countPaths(maze, x, y + 1, visited, count,templist,dataframe)
 
        # go left `(x, y)` ——> `(x, y - 1)`
        if y - 1 >= 0 and not visited[x][y - 1]:
            count = countPaths(maze, x, y - 1, visited, count,templist,dataframe)
            
    #paths that end in mines,d
    if isValidCell(x, y, maze) and maze[x][y]==0:
        dataframe.append(['d'])
        count_mine+=1
        
    #paths blocked, l  
    if isValidCell(x, y, maze) and maze[x][y]==1 and (x,y)!=(N-1,N-1):
        if ((not isValidCell(x+1, y, maze) or visited[x+1][y]) and (x+1,y)!=(N-1,N-1)) and ((not isValidCell(x-1, y, maze) or visited[x-1][y])and (x-1,y)!=(N-1,N-1)) and ((not isValidCell(x, y+1, maze) or visited[x][y+1])and (x,y+1)!=(N-1,N-1)) and ((not isValidCell(x, y-1, maze) or visited[x][y-1])and (x,y-1)!=(N-1,N-1)):
            dataframe.append(['l'])
            count_block+=1
                
        
     
    # backtrack from the current cell and remove it from the current path
    visited[x][y] = False
    #templist.remove([x,y])
    #print("removed", x,y)
    return count

# Main 

In [83]:
if __name__ == '__main__':
       
    #m_list = list(set(mine_num_list))
    #s_num, d_num, l_num, total_num = [], [], [], []
    exp_s_num,exp_d_num,exp_l_num=[],[],[]

    templist = []
    for m in m_list:
        mazelist = grids(n,m)
        count_s, count_d, count_l = 0, 0, 0
        explist=[]
        for maze in mazelist:

            dataframe = []
            N = len(maze)
 
            count1 = 0
            count_mine = 0
            count_block = 0
 
            visited = [[False for x in range(N)] for y in range(N)]
 
            #countlistwashere
            
            #exp_grids is the list of grids generated over the 1000 runs in the first cell when trying to find number of mines
            #for a given p
            
            for i in exp_grids:
                comparison = maze == i
                equals= comparison.all()
                if equals:
                    count1 = countPaths(maze, 0, 0, visited, count1,templist,dataframe)
                    explist.append(random.choice(dataframe))
            
            '''if mazelistcount == mazechoice:
            #here we chose the dataframe from the maze chosen above and take a random path from the df( thats the df.sample() method)
                k=dataframe.sample().reset_index(drop=True)
                explist.append(k.at[0, 'path label'])'''
        
            #count_s+=count1
            #count_d+=count_mine
            #count_l+=count_block
        
        #for m, num of chosen paths is equal to frequency of m 
        
        
 
        #print("Number of ideal paths for a" , n,"x",n , "grid with" , m , "mine(s):")
        #print("The number of safe paths are", count_s)
        #s_num.append(count_s)
        #print("The number of paths that hit mines are", count_d)
        #d_num.append(count_d)
        #print("The number of blocked paths are", count_l)
        #l_num.append(count_l)
        #print("The total number of paths are", count_s + count_d + count_l , "\n")
        
        #total_num.append(count_s + count_d + count_l)
    #get the counts of exprimental data for each m
    
        explist = np.array(explist).flatten()

        dicti=col.Counter()
        dicti=col.Counter(explist)
        exp_s_num.append(dicti['s'])
        exp_d_num.append(dicti['d'])
        exp_l_num.append(dicti['l'])
        
        
        print(dicti['s'] , dicti['d'] , dicti['l'] , dicti['s']+dicti['d']+dicti['l'])
   
    

0 39 0 39
0 104 0 104
0 26 0 26
0 346 0 346
0 251 0 251


**uncomment and run below cell if you want to verify listall**

In [77]:
# uncomment and run this just to verify listall  
'''for j in range (0,len(explist)):
    dicti=col.Counter()
    for sublist in explist[j]:
        dicti.update(sublist)
    print(dicti)
'''

'for j in range (0,len(explist)):\n    dicti=col.Counter()\n    for sublist in explist[j]:\n        dicti.update(sublist)\n    print(dicti)\n'

In [78]:
#expectation

E1, E2 = 0, 0

#can be changed based on rv system
#-----------------#

s, l, d = 2, 1, 0

#-----------------#

#min_freq[i] is P(A_i) where i is m ; x_num[i] is the number of paths (s or l or d) ; total_num[i] is the total number of paths for a given i (=m)
#x_num[i]/total_num[i] = P(x|A_i)

for i in range(len(exp_s_num)):
    E1+= (s*(exp_s_num[i]/ite))+(l*(exp_l_num[i]/ite))+(d*(exp_d_num[i]/ite))
    #E2+= mine_freq[i]/1000*((s*(s_num[i]/total_num[i]))+(l*(l_num[i]/total_num[i]))+(d*(d_num[i]/mine_freq[i])))

print("Expectation for p=" , str(prob))
print(E1)

Expectation for p= 0.9
0.0
